# Mašinsko učenje

Umesto priče o tome šta je mašinsko učenje, jedna slika govori hiljadu reči ;)

![img/ml_one_slide.jpg](img/ml_one_slide.jpg)

Dva najčešće korišćena tipa mašinskog učenja su:
* Nadgledano učenje (*eng. supervised learning*)
* Nenadgledano učenje (*eng. unsupervised learning*)

---

## Nadgledano učenje

Učenje funkcije na osnovu obeleženih podataka, odnosno predviđanje *y* u odnosu na *x* iz primera *(x,y)* (setite se linearne regresije). Performanse postupka se mere određenom metrikom/greškom. Veliki broj postupaka, neki od najpopularnijih su:
* Veštačke neuronske mreže
* Stabla odlučivanja + Random forests
* Naivni Bayes klasifikator
* kNN (k nearest neighbors)
* SVM (support vector machine)
* ... i još mnogo drugih ...

*Više o nadgledanom učenju sledeći čas!*

---

## Nenadgledano učenje

Učenje funkcije koja opisuje "skrivenu" strukturu neobeleženih podataka, odnosno učenje reprezentacije podataka na osnovu primera *x* (nema *y*). Nenadgledano učenje može biti korisno za pronalaženje interesantnih veza među podacima. Ne postoji neki standardni način merenja performansi. Postupci se dele na:
* Klasterizacija/klasterovanje (*eng. clustering*) - grupisanje podataka na osnovu sličnosti
* Analiza komponenti (*eng. component analysis*) - otkrivanje najdeskriptivnikih osobina podataka
* Asocijativna pravila (*eng. association rules*) - pronalaženje uobičajenih kombinacija osobina podataka

---


### K-means

Jedan od najčešće korišćenih algoritama za nenadgledano klasterovanje podataka. Preciznije, k-means je ne-hijerarhijska metoda grupisanja sličnih podataka. K-means je tehnika koja se često koristi u tzv. *eksplorativnoj analizi podataka*.

Klaterizacija je zadatak grupisanja skupa objekata, tako da su objekti koji su u istoj grupi (odnosno *klasteru*) sličniji (u nekom smislu) jedni drugima, više nego što su slični objektima u drugim grupama (klasterima).

Pseudo-kod:

```
za svaku grupu inicijalizovati nasumično centar
dok se centri ne prestanu kretati ili ne dostigne max broj iteracija:
    pridruži svaki element grupi sa njemu najbližim centrom grupe
    pomeri centar svih grupa na osnovu novih elemenata
```

![img/kmeans.gif](img/kmeans.gif)

** TODO 1: ** Implementirati K-means algoritam (u ```src/kmeans.py``` sve što je označeno sa TODO 1):
  * Metodu ```fit``` klase ```KMeans```, koja kao parametar prima n-dimenzionalne podatke koje treba klasterizovati.
  * Metodu ```predict``` klase ```KMeans```, koja kao parametar prima jedan podatak i vraća indeks klastera kojem pripada (na osnovu euklidske udaljenosti).
  * Metodu ```recalculate_center``` klase ```Cluster```, koja treba da izračuna centar klastera kao srednju vrednost svih podataka u njemu i da vrati ovu vrednost kao rezultat.
  * ** Za domaći ** - proširiti metodu ```fit``` tako da se algoritam zaustavi kada se centri klastera više ne pomeraju.
  
** ТОDO 2: ** Primeniti K-means algoritam na Iris data setu (https://en.wikipedia.org/wiki/Iris_flower_data_set), u cilju pronalaženja dva klastera u odnosu na širinu sepala i dužinu petala. Pokrenuti skriptu ```src/iris.py```.


![img/iris.png](img/iris.png)


#### Određivanje optimalnog K

Kako znati unapred koliko ima klastera? Lako je videti kada su podaci dvodimenzionalni, jer ih je onda lako i vizualizovati, ali često podaci imaju (mnogo) više od samo 2 dimenzije - ovo je tzv. "kletva dimenzionalnosti" (*eng. curse of dimensionality*) u mašinskom učenju.

Određivanje optimalnog K (tj. broja klastera) je nešto se dosta proučavalo, a mi ćemo koristi tzv. "metodu lakta" (*eng. elbow method*). Za određen broj K (npr. 2, 4, 6, 8, ..., 20) se vrši klasterizacija i zatim se računa suma kvadratnih grešaka (SSE). SSE se računa tako što se unutar svakog klastera sumiraju kvadrati udaljenosti podataka od centra klastera, i zatim se sve to opet sumira. Matematički:

$ SSE = \sum_{i=1}^{K} \sum_{x \in c_{i}} dist(x, c_{i})^{2} $, gde je *dist* euklidska udaljnost.

Zatim se za sve plotuje SSE u odnosu na K, npr.:

![img/sse.png](img/sse.png)

** TODO 3: ** Implementirati izračunavanje sume kvadratnih grešaka (SSE) u metodi ```sum_squared_error``` klase ```KMeans```. Zatim ponovo pokrenuti ```src/iris.py``` skriptu i odrediti optimalno K na osnovu grafika.

** TODO 4: ** Normalizovati podatke pre primene K-means algoritma. Normalizacija je potrebna kada su opsezi vrednosti u podacima različiti (npr. ako se visina ljudi meri u metrima, a težina u gramima). Normalizacija se može vršiti na više načina, dva najčešća su:
* Svođenje podataka na isti interval, npr. [-1, 1]. Šta je neophodno znati o podacima za ovakvu normalizaciju?
* Centriranje i normalizacija standardnom devijacijom -> x = (x - srednja_vrednost(x)) / standardna_devijacija(x). Ovakva normalizacije se najčešće koristi i nije karakteristična samo za K-means, već generalno za analizu podataka u mašinskom učenju.

** TODO 5: ** Primeniti K-means na podatke generisane u ```src/ball_circle.py```.

---

#### Prednosti K-means

* Jednostavan i lako razumljiv
* Laka implementacija
* Relativno dobre performanse (za malo K)
* Odličan kada su klasteri sferičnog/globularnog oblika (malo formalnije hiper-sferičnog, za sfere u >3 dimenzija)

#### Mane K-means
* Potrebno unapred znati K (što je nekad teško odrediti)
* Nije deterministički - pošto se centri inicijalizuju nasumično, nekad se dobijaju drugačiji rezultati
* Osetljiv na šum
* Kada podaci nisu globularnog oblika -> beskoristan (pogledati donju sliku)
* Nema mogućnost hijerarhijskog klasterovanja (razlikovanje više manjih podklastera unutar većeg klastera)

![img/kmeans_fail.png](img/kmeans_fail.png)

---


### DBSCAN

DBSCAN (Density-based spatial clustering of applications with noise) je takođe algoritam za klasterizaciju podataka. Ovaj postupak se zasniva na ideji grupisanja tačaka (podataka) na osnovu njihove međusobne udaljenosti. Ukoliko se tačke nalaze u tzv. *epsilon okolini* one su deo nekog klastera, u suprotnom se posmatraju kao šum.

Iako predstavljen još 1996. godine, 2014. godine na jednoj od najprestižnijih konferencija (KDD), DBSCAN je nagrađen nagradom "testa vremena", kao jedan od najviše citiranih algoritama sa ogromnom upotrebom, kako u teoriji tako i u praksi.


Opis DBSCAN algoritma:
1. Neka postoji neki skup tačaka (podataka) koje želimo da klasterizujemo. U samom postupku, razlikuju se tri vrste tačaka: ključne tačke, dostupne tačke i šum.
2. DBSCAN zahteva dva parametra: *epsilon* (eps) i *minimalni broj potrebnih tačaka koje čine region* (minPts). Epsilon okolina se najčešće računa korišćenjem euklidske udaljenosti.
3. Algoritam počinje sa proizvoljnom tačkom. Računa se epsilon okolina te tačke, i ukoliko se u njoj nalazi dovoljno tačaka (minPts), započinje se novi klaster. U suprotnom, tačka se računa kao šum. Obratiti pažnju da tačka, iako je šum, kasnije *može* biti pronađena kao deo neke druge epsilon okoline sa dovoljno tačaka i samim tim da postane deo klastera.
4. Ukoliko je za tačku određeno da pripada klasteru, sve tačke u njenoj epsilon okolini takođe pripadaju tom klasteru. Dakle, sve tačke koje su pronađene u epsilon okolini trenutne tačke se dodaju u klaster, kao i tačke koje se nalaze u epsilon okolini tih tačaka (rekurzivno). Proces se nastavlja dok se ne nađe ceo klaster, odnosno dok se ne obiđu sve tačke u epsilon okolinama. 
5. Onda se nalazi nova, prozivoljna neposećena tačka, za koju se ponavlja čitav postupak, što dovodi do otkrivanja ili novog klastera ili šuma.

![img/dbscan.png](img/dbscan.png)

Pseudo-kod algoritma (preuzet sa Wikipedia stranice):

```
DBSCAN(D, eps, MinPts) {
   C = 0
   for each point P in dataset D {
      if P is visited
         continue next point
      mark P as visited
      NeighborPts = regionQuery(P, eps)
      if sizeof(NeighborPts) < MinPts
         mark P as NOISE
      else {
         C = next cluster
         expandCluster(P, NeighborPts, C, eps, MinPts)
      }
   }
}

expandCluster(P, NeighborPts, C, eps, MinPts) {
   add P to cluster C
   for each point P' in NeighborPts { 
      if P' is not visited {
         mark P' as visited
         NeighborPts' = regionQuery(P', eps)
         if sizeof(NeighborPts') >= MinPts
            NeighborPts = NeighborPts joined with NeighborPts'
      }
      if P' is not yet member of any cluster
         add P' to cluster C
   }
}

regionQuery(P, eps)
   return all points within P's eps-neighborhood (including P)
```

** TODO 6: ** Implementirati DBSCAN algoritam u metodi ```fit``` klase ```DBSCAN```, ```src/dbscan.py```.

** TOOD 7: ** Primeniti DBSCAN nad Iris podacima ```src/iris.py``` i nad podacima u skripti ```src/ball_circle.py```.

---

#### Prednosti DBSCAN

* Nije potrebno unapred znati broj klastera (kao kod K-means)
* Klasteri mogu biti proizvoljnog oblika
* Ume da tretira šum
* Parametre epsilon i minPts je lako menjati u cilju dobijanja klastera različitih veličina i oblika, i ove parametre često podešavaju eksperti sa domenskim znanjem


#### Mane DBSCAN

* Kvalitet rezultata zavisi od toga čime se meri epsilon. Obično je to euklidska udaljenost, ali za višedimenzionalne podatke potrebne su drugačije metrike
* Kada postoje varijacije u gustini klastera, nemoguće je odrediti epsilon i minPts da odgovara svim klasterima
* U slučaju kada ne postoji ekspert sa domenskim znanjem, određivanje epsilon i minPts parametara je često dosta teško


---

** DODATNO **:

* Izvršiti klasterizaciju u cilju eksplorative analize podataka nad bankovnim podacima ```data/bank.csv```. (4 boda)
* Izvršiti klasterizaciju nad data setovima koji se nalaze na linku https://people.sc.fsu.edu/~jburkardt/datasets/hartigan/hartigan.html (3 boda).
